<a href="https://colab.research.google.com/github/mars-ef/Trustpilot_Scraper/blob/main/Trustpilot_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Analytics with Adam video on how to use https://www.youtube.com/watch?v=VD0wIiLVSRA
#Original code: https://github.com/analyticswithadam/Python/blob/main/Web_Scraping_from_TrustPilot.ipynb

from time import sleep
import requests
from google.colab import files
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def soup2dict(src, data, key, attr=None):
    if attr:
        for i, val in enumerate(src):
            data[i][key] = val[attr]
    else:
        for i, val in enumerate(src):
            data[i][key] = val.get_text()

data = []
from_page = 1
to_page = 1
company = 'chase.co.uk' #E.g. for https://www.trustpilot.com/review/wise.com to sort default take 'wise.com'. To filter by most recent include 'wise.com?sort=recency'

for i in range(from_page, to_page + 1):
    result = requests.get(f"https://www.trustpilot.com/review/{company}?page={i}") #for most recent change to '{company}page={i}&sort=recency'
    soup = BeautifulSoup(result.content, 'html.parser')

    for e in soup.select('article'):
        reply_date_container = e.select_one('.styles_replyInfo__FYSje')
        if reply_date_container:
            reply_date_element = reply_date_container.find('time')
            if reply_date_element:
                reply_date = reply_date_element.get('datetime').split('T')[0]
            else:
                reply_date = None
        else:
            reply_date = None

        data.append({
              'brand': "Chase", #Change to your brand
              'username':e.select_one('[data-consumer-name-typography]').text,
              'location':e.select_one('[data-consumer-country-typography] span').text,
              'number_of_reviews': e.select_one('[data-consumer-reviews-count]').text.split(' ')[0] if e.select_one('[data-consumer-reviews-count]') else None,
              'review_rating': e.select_one('[data-service-review-rating] img').get('alt').split(' ')[1],
              'date':e.select_one('[data-service-review-rating] time').get('datetime').split('T')[0],
              'invited':e.select_one('[data-review-label-tooltip-trigger] span').text if e.select_one('[data-review-label-tooltip-trigger] span') else None,
              'review_title':e.h2.text,
              'review_text':e.select_one('[data-service-review-text-typography]').text if e.select_one('[data-service-review-text-typography]') else None,
              'date_of_experience': pd.to_datetime(e.select_one('[data-service-review-date-of-experience-typography]').text.split(': ')[-1]),
              'reply_from_business':e.select_one('[data-service-review-business-reply-text-typography]').text if e.select_one('[data-service-review-business-reply-text-typography]') else None,
              'reply_date': reply_date,
              'page_number': i #useful for debugging
              })

    #To prevent throttling
    sleep(1)

    # Create the DataFrame
    review_data = pd.DataFrame(data)

In [2]:
review_data

,brand,username,location,number_of_reviews,review_rating,date,invited,review_title,review_text,date_of_experience,reply_from_business,reply_date,page_number
0,Chase,Sam Downey,GB,8,5,2024-09-30,None,Great service,Great service. Easy to use app and with the ex...,2024-09-29,Thank you for taking the time to review!,2024-09-30,1
1,Chase,Mr Adrian Robinson,GB,19,5,2024-09-30,Redirected,My online Chase account,I opened the online account & it was hassle fr...,2024-09-30,Thank you for sharing your positive experience...,2024-09-30,1
2,Chase,Mr Frimpong,GB,21,5,2024-10-01,Redirected,Great service,Great service. I have had one or two initial m...,2024-09-19,None,None,1
3,Chase,Mark Eastbourne,GB,20,5,2024-09-30,Redirected,Really glad I switched,This is an excellent account if you are happy ...,2024-09-30,Thank you for sharing your experience with us!,2024-10-01,1
4,Chase,Alison,GB,7,5,2024-09-29,Redirected,Excellent Well Informed Advisor,Very impressed with fact the advisors all seem...,2024-09-28,We love to hear you are a happy customer!,2024-09-30,1
5,Chase,Lisa Terry,GB,12,5,2024-09-30,None,Understanding and helpful bank,After watching the difficulties some people ha...,2024-09-30,Thank you for such a fantastic review!,2024-10-01,1
6,Chase,David Carpenter,GB,21,5,2024-09-29,Redirected,Chase Bank,The app is easily understood and a joy to use....,2024-09-29,Thank you for your kind words and recommendati...,2024-09-30,1
7,Chase,David cox,GB,15,5,2024-09-25,Redirected,On line banking that works,Really great on line banking system . Bit diff...,2024-09-25,Thanks for the feedback!,2024-09-27,1
8,Chase,CW,GB,3,5,2024-09-26,None,Great service and a fabulous app.,Great service and a fabulous app. So easy to u...,2024-09-26,"Thank you for sharing, we appreciate the kind ...",2024-09-27,1
9,Chase,Chris Lee,GB,54,5,2024-09-26,None,A good bank shame no branches in the UK,Have been banking with Chase since they starte...,2024-09-26,Thank you for sharing!,2024-09-27,1


In [ ]:
# Convert the DataFrame to a CSV file and download it
review_data.to_csv('reviews.csv', index=False)
files.download('reviews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>